<center>

# Discrete Event Systems

## Home Assignment 2

### Nonblocking and Controllable Supervisor Synthesis

- - -

# Introduction

The objective of this home assignment is to extend the results already obtained in Home Assignment 1 by including uncontrollable events and generating a supervisor that is both nonblocking and controllable. This will be achieved by implementing two algorithms already presented in the lecture notes in Python, but also implementing a new function that generates uncontrollable states.

This home assignment is performed in *two member groups*. Write all your answers into this notebook and **submit only this notebook (.ipynb) on Canvas**

## Comments and Recommendations
Once again:
* The following resources will be of great help to you for this assignment:
 * Lecture notes
 * [Python docs](https://docs.python.org/3/)
 * [Google](https://www.google.com)
 * [Wikipedia overview of syntax and semantics](https://en.wikipedia.org/wiki/Python_syntax_and_semantics)
* This assignment is written for Python 3.5 or later!
* If you get stuck with Python or Jupyter, try to google it! E.g. your search query could be *"python --INSERT ERROR HERE--"*. Chances are high, heaps of people had the same problem before you.
* If your code passes all the tests in this notebook, you are on a good way. That does not mean you have passed though! We will test your code with additional edge cases. So convince yourself that everything is correct before you submit.

- - -

# Task 1

## Reachability v.2 *[2p]*

Before we can turn to the synthesis of non-blocking and controllable supervisors, we need to update our `reach` function to handle uncontrollable events and forbidden states too.

As defined in chapter 6 of the lecture notes the reachability algorithm is as follows:

**Algorithm 1.** Reachability($\Sigma, \delta, Q_0, Q_x$)

>**let** $k:=0, Q_0 := Q_0 \setminus Q_x$
>
>**repeat**
>
>>$k := k+1$
>>
>>$Q_k := Q_{k-1} \cup \{q^+ \mid \exists q \in Q_{k-1} \wedge \exists \sigma \in \Sigma \wedge \delta(q,\sigma) = q^+ \} \setminus Q_x $
>>
>**until** $Q_k = Q_{k-1}$
>
**return** $Q_k$

In this assignment, we will re-use parts of Assignment 1. Make sure to execute the cell below.

In [1]:
from util import Transition
from util import plot_automaton
from util import filter_trans_by_source, filter_trans_by_events, filter_trans_by_target
from util import extract_elems_from_trans, flip_trans

In [20]:
def reach(events, trans, start_state, forbidden_States):
    """
    Returns the forward reachable states of a transition set
    
    :param trans: set of transitions
    :param start_state: set of states
    """
    # YOUR CODE HERE
    # raise NotImplementedError()
    reachable_states = set()
    states_to_explore = set()
    forbid_stat = set(forbidden_States)
    start = start_state.copy()

    if start.intersection(forbidden_States):
        return reachable_states
    else:
        reachable_states.update(start_state)
        states_to_explore.update(start_state)

    while states_to_explore:
    
        current_state = states_to_explore.pop()
        for transition in trans:
            
            if transition.source == current_state and transition.target not in reachable_states and transition.target not in forbid_stat: 
                reachable_states.add(transition.target)
                states_to_explore.add(transition.target)
    return reachable_states

In [9]:
#Space for your tests

In [21]:
# Single state
reachable = reach(set(), set(), {1}, set()) 
assert reachable == {1}, 'Got {} instead'.format(reachable)
reachable = reach({'a'}, {Transition(1, 'a', 1)}, {1}, set())
assert reachable == {1}, 'Got {} instead'.format(reachable)

# Simple automata
reachable = reach({'a'}, {Transition(1, 'a', 2)}, {1}, set())
assert reachable == {1, 2}, 'Got {} instead'.format(reachable)
reachable = reach({'a'}, {Transition(2, 'a', 1)}, {1}, set())
assert reachable == {1}, 'Got {} instead'.format(reachable)
reachable = reach({'a'}, {Transition(2, 'a', 1), Transition(1, 'a', 2)}, {1}, set())
assert reachable == {1, 2}, 'Got {} instead'.format(reachable)

# Parallel transitions
reachable = reach({'a', 'b'}, {Transition(1, 'a', 2), Transition(1, 'b', 2)}, {1}, set())
assert reachable == {1, 2}, 'Got {} instead'.format(reachable)
reachable = reach({'a', 'b'}, {Transition(2, 'a', 1), Transition(2, 'b', 1)}, {1}, set())
assert reachable == {1}, 'Got {} instead'.format(reachable)

# Handling of forbidden states
reachable = reach({'a'}, {Transition(1, 'a', 2)}, {1}, {1})
assert reachable == set(), 'Got {} instead'.format(reachable)
reachable = reach({'a'}, {Transition(1, 'a', 2)}, {1}, {2})
assert reachable == {1}, 'Got {} instead'.format(reachable)

# Larger automata
reachable = reach({'a'},
                  {Transition(1, 'a', 2), Transition(2, 'a', 3), Transition(3, 'a', 1)},
                  {1},
                  set())
assert reachable == {1, 2, 3}, 'Got {} instead'.format(reachable)

# Preliminaries

For the remainder of the assignment use the functions given below.

In [22]:
from util import Automaton
from util import merge_label, cross_product 


def coreach(events, trans, start_states, forbidden):
    """
    Returns the coreachable (backward reachable) states of a transition set
    
    :param events: set of events
    :param trans: set of transitions
    :param start_states: set of states
    :param forbidden: set of forbidden states
    """
    return reach(events, flip_trans(trans), start_states, forbidden)


def synch(aut1, aut2):
    """
    Returns the synchronous composition of two automata.
    
    :param aut1: Automaton
    :param aut2: Automaton
    """
    states = cross_product(aut1.states, aut2.states)
    init = merge_label(aut1.init, aut2.init)
    events = aut1.events | aut2.events
    
    # Add self loops
    trans1 = aut1.trans.copy()
    for event in aut2.events - aut1.events:
        for state in aut1.states:
            trans1.add(Transition(state, event, state))
    
    trans2 = aut2.trans.copy()
    for event in aut1.events - aut2.events:
        for state in aut2.states:
            trans2.add(Transition(state, event, state))
    
    # Transitions
    trans = set()
    for t1 in trans1:
        for t2 in trans2:
            if t1.event == t2.event:
                trans.add(Transition(merge_label(t1.source, t2.source),
                                     t1.event,
                                     merge_label(t1.target, t2.target)))
    
    # Marked states
    m1 = aut1.marked if aut1.marked else aut1.states
    m2 = aut2.marked if aut2.marked else aut2.states
    marked = cross_product(m1, m2)
    
    # Forbidden states
    forbidden = cross_product(aut1.forbidden, aut2.states) | cross_product(aut1.states, aut2.forbidden)
    
    # Removing unreachable states
    reachable = reach(events, trans, {init}, set())
    marked &= reachable
    forbidden &= reachable
    trans = filter_trans_by_source(trans, reachable)
    
    # Create synchronized automaton
    return Automaton(reachable, init, events, trans, marked, forbidden)

- - -

# Task 2

## Synthesis *[3p]*

Using `reach`, `coreach` and `synch`, your task is now to implement:

$\qquad$ `S = supervisor(P, Sp, sigma_u)`
    
that, given a plant $P$, a specification $Sp$ and a set of uncontrollable events $\Sigma_u$, generates a nonblocking and controllable supervisor $S$. Note that $\Sigma_u \subseteq \Sigma^P$, and it is assumed that $\Sigma^{Sp} \subseteq \Sigma^P$, where $\Sigma^P$ and $\Sigma^{Sp}$ are the event sets of the plant $P$ and the specification $Sp$. The supervisor is generated by implementing and applying algorithm 3 in the lecture notes.

Your code should raise a `ValueError` when there exist no supervisor that can fulfill the specification:

$\qquad$ `if noSupervisorExists: raise ValueError`

* *Note:* If more than one plant and/or specification model are involved, the total plant and/or specification model is generated by synchronizing the local models. For instance, $P_1, P_2, Sp_1$ and $Sp_2$ gives $P = P_1 \parallel P_2$ and $Sp = Sp_1 \parallel Sp_2$.

* *Note:* One of the input sets in Algorithm 3 is the set of forbidden states. Implement specifically a function that generates this forbidden state set, including both explicitly forbidden states and uncontrollable states.

* *Note:* In the synchronization of two automaton $A$ and $B$, the names of the original states $q^A$ and $q^B$ can be identified, since the merge state function generates state names $q^A.q^B$. Identification of the reachable states in $P \parallel Sp$ and the corresponding states in $P$ and $Sp$ simplifies the generation of the uncontrollable states. 

Here are some helper functions that might help you with your implementation of the algorithm in code.

In [13]:
# These two functions follow the definition of uncontrollable states given on page 100 in the Lecture Notes
def is_defined_for_p(p_trans, merged_source, shared_sigma_u):
    """
    Boolean check whether there is an uncontrollable transition defined in P
    with the merged_source as start state.
    
    :param p_trans: Set of transitions in automaton P
    :param merged_source: String label of the merged state in P||Q
    :param shared_sigma_u: Set of uncontrollable events shared between P and Q.
    """
    validate_inputs(p_trans, merged_source, shared_sigma_u)
    uncontrollable_trans = filter_trans_by_events(p_trans, shared_sigma_u)
    u_trans_from_merged_source = {t for t in uncontrollable_trans 
                                  if merged_source.startswith(str(t.source))}
    return u_trans_from_merged_source != set()


def is_defined_for_q(q_trans, merged_source, shared_sigma_u):
    """
    Boolean check whether there is an uncontrollable transition defined in Q
    with the merged_source as start state.
    
    :param q_trans: Set of transitions in automaton Q
    :param merged_source: String label of the merged state in P||Q
    :param shared_sigma_u: Set of uncontrollable events shared between P and Q.
    """
    validate_inputs(q_trans, merged_source, shared_sigma_u)
    uncontrollable_trans = filter_trans_by_events(q_trans, shared_sigma_u)
    u_trans_from_merged_source = {t for t in uncontrollable_trans 
                                  if merged_source.endswith(str(t.source))}
    return u_trans_from_merged_source != set()


def validate_inputs(trans, source, events):
    assert type(trans) is set
    assert type(source) is str
    assert type(events) is set

In [25]:
# Now implement the function in code
def supervisor(P, Sp, sigma_u):
    """
    Generates a nonblocking and controllable supervisor for the synchronized system P||Sp.
    
    :param P: automaton of the plant
    :param Sp: automaton of the specification
    :param sigma_u: set of uncontrollable events
    """
    # YOUR CODE HERE
    Coreach_p = coreach(P.events, P.trans,p.init,p.forbidden)
    Coreach_sigma = coreach(sigma_u,P.trans, P.states - Coreach_p,{})
    
    
    Coreach_p = coreach(P.events, P.trans,p.init,p.forbidden)
    Coreach_sigma = coreach(sigma_u,P.trans, P.states - Coreach_p,{})
    S  = synch(P,Sp)
    
    # raise NotImplementedError()
    return S

In [ ]:
# Space for your own tests

In [24]:
# Test case 0 - Single states
P = Automaton(states={1},
              init=1,
              events=set(),
              trans=set())
Sp = Automaton(states={2},
               init=2,
               events=set(),
               trans=set())
S = supervisor(P, Sp, set())
assert S.states == {'1.2'}, 'TC 0.1 - got {} instead'.format(S.states)
assert S.events == set(), 'TC 0.2 - got {} instead'.format(S.events)

# Test case 1 - Three states
P = Automaton(states={1, 2, 3},
              init=1,
              events={'a', 'b'},
              trans={Transition(1, 'a', 2), Transition(2, 'b', 3)})
Sp = Automaton(states={1, 2},
               init=1,
               events={'a'},
               trans={Transition(1, 'a', 2)})
S = supervisor(P, Sp, set())
assert S.states == {'1.1', '2.2', '3.2'}, 'TC 1.1 - got {} instead'.format(S.states)
assert S.init == '1.1', 'TC 1.2 - got {} instead'.format(S.init)
assert S.events == {'a', 'b'}, 'TC 1.3 - got {} instead'.format(S.events)
assert S.forbidden == set(), 'TC 1.4 - got {} instead'.format(S.forbidden)

# Test case 2 - Three states and an uncontrolable event
P = Automaton(states={1, 2, 3},
              init=1,
              events={'a', 'b'},
              trans={Transition(1, 'a', 2), Transition(2, 'b', 3)})
Sp = Automaton(states={1, 2},
               init=1,
               events={'a', 'b'},
               trans={Transition(1, 'a', 2)})
S = supervisor(P, Sp, {'b'})
assert S.states == {'1.1'}, 'TC 2.1 - got {} instead'.format(S.states)
assert S.init == '1.1', 'TC 2.2 - got {} instead'.format(S.init)
assert S.events == {'a', 'b'}, 'TC 2.3 - got {} instead'.format(S.events)
assert S.forbidden == set(), 'TC 2.4 - got {} instead'.format(S.forbidden)

# Test case 3 - Two uncontrollable transitions in a sequence
P = Automaton(states={1, 2, 3, 4},
              init=1,
              events={'a', 'b'},
              trans={Transition(1, 'a', 2), Transition(2, 'b', 3), Transition(3, 'b', 4)})
Sp1 = Automaton(states={1, 2, 3},
                init=1,
                events={'a', 'b'},
                trans={Transition(1, 'a', 2), Transition(2, 'b', 3)},
                marked={1, 2}) 
Sp2 = Automaton(states={1, 2, 3, 4},
                init=1,
                events={'a', 'b'},
                trans={Transition(1, 'a', 2), Transition(2, 'b', 3), Transition(3, 'b', 4)},
                marked={1, 2})
S = supervisor(P, Sp1, {'b'})
assert S == supervisor(P, Sp2, {'b'})
assert S.states == {'1.1'}, 'TC 3.1 - got {} instead'.format(S.states)
assert S.init == '1.1', 'TC 3.2 - got {} instead'.format(S.init)
assert S.events == {'a', 'b'}, 'TC 3.3 - got {} instead'.format(S.events)
assert S.trans == set(), 'TC 3.4 - got {} instead'.format(S.trans)
assert S.forbidden == set(), 'TC 3.4 - got {} instead'.format(S.forbidden)

NameError: name 'p' is not defined

---

# Task 3

Evaluate the implementation in Task 2 by generating a nonblocking and controllable supervisor $S$ for the plant and specification models given in 

* Example 7.5 on p. 113 in the Lecture Notes. **[1p]**

* Example 7.6 on p. 114 in the Lecture Notes. **[1p]**

* Task 3 in the Exam Oct 24, 2015 (defined below). **[1p]**

## Example 7.5

In [ ]:
# Create automata P1, P2, SP1 and SP2
# YOUR CODE HERE
raise NotImplementedError()

Sp = synch(SP1, SP2)
P = synch(P1, P2)

S = supervisor(P, Sp, {'b'})
plot_automaton(S, 'S_7dot5')

In [ ]:
assert type(S) is Automaton

# This is optional but simplifies the testing
p1_states = {'p11', 'p12'}
p2_states = {'p21', 'p22'}
sp1_states = {'sp11', 'sp12'}
sp2_states = {'sp21', 'sp22'}
events = {'a', 'b', 'c', 'd', 'e'}
assert all({s[0:3] in p1_states and s[4:7] in p2_states and s[8:12] in sp1_states and s[13:17] in sp2_states 
            for s in S.states}), 'This is optional but, to simplify the tests, \
            please use only the state name from the example and synchronize in the same order, \
            i.e init state would be p1.p2.sp1.sp2'
assert all({e in events for e in S.events}), 'This is optional but, to simplify the tests, \
            please use only the event labels from the example.'

## Example 7.6

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

S = supervisor(P, Sp, {'d'})
plot_automaton(S, 'S_7dot6')

In [ ]:
assert type(S) is Automaton

# These tests are optional but simplify the hidden tests
p_states = {'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8'}
q_states = {'q1', 'q2', 'q3', 'q4', 'q5', 'q6', 'q7'}
events = {'a', 'b', 'c', 'd'}
assert all({s.startswith(tuple(p_states)) and s.endswith(tuple(q_states)) for s in S.states}), '\
    This is optional but, to simplify the tests, please use only the state name from the example \
    and synchronize in the same order, i.e init state would be p1.q1'
assert all({e in events for e in S.events}), '\
    This is optional but, to simplify the tests, please use only the event labels from the example.'

## Exam Oct 2015 - Task 3

Consider a plant $P$ with the language
<center>
$L(P) = \overline{ac(b + d) + bc(a + ecd) + cd},$
</center>
and a specification $Sp$ given by the marked language
<center>
$L_m(Sp) = ab + ba + d.$
</center>
Assume that the events $d$ and $e$ are uncontrollable, while $a$, $b$ and $c$ are controllable.

* Formulate automata for the languages $L(P)$ and $L_m(Sp)$ with minimal number of states.
* Generate a controllable and nonblocking supervisor.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

S = supervisor(P, Sp, {'d', 'e'})
plot_automaton(S, 'S_3_15')

In [ ]:
assert type(S) is Automaton

---

# Task 4 *[2p]*

Two people named **A** and **B** are playing a simple game. A number of sticks are lain
out on the ground and the players take alternately one or two sticks. Note that at least
one stick must be picked. The player that ends up with the last stick has lost the game.
Player **A** is always the one that starts picking sticks. A related problem is introduced
in Task 6.3 in the DES-exercises.

* Model this game by an automaton `P`, with an initial number of five sticks. Hint: identify the events and the states.

* Create a specification `Sp` by introducing a marked state in `P`, specifying that player **A** is to win and player **B** is to loose. Remember that the player left with only the final stick to pick, is the loser.

* Generate by the algorithm in *Task 1* a supervisor `S` which guarantees that player **A** wins the game, i.e `S = supervisor(P, Sp, sigma_u)`. *Note* that the set of uncontrollable events `sigma_u` must first be decided.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

S = supervisor(P, Sp, sigma_u)
plot_automaton(S, 'S_3_15')

In [ ]:
assert type(P) is Automaton
assert type(Sp) is Automaton
assert type(S) is Automaton

assert P.states == Sp.states
assert P.events == Sp.events
assert P.init == Sp.init
assert P.trans == Sp.trans
assert P.marked == set() or P.marked == P.states
assert len(Sp.marked) == 1

- - -

Great! You have reached the bottom of the page. If you are done,

* Save the notebook
* Upload the .ipynb file to Canvas
* Go for fika! 